In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import scipy as sp
from scipy import stats
import scipy.stats.mstats as mst
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import (pairwise_tukeyhsd, MultiComparison)

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

data = pd.read_csv('901_CleanedData.csv', low_memory=False)

In [ ]:
def newIV(row):
    if row['DISC_SCORE0'] < 2:
        val = 0
    else:
        val = 1
    return val

data['DISC'] = data.apply(newIV, axis=1)

Factors: DISC, RACE, DEGREE, INCOME0, CVD

Continuous: CRPRESU0, BMI0

In [ ]:
formula = 'LV_AGE_DIFF0 ~ C(DISC) + C(RACE) + C(DEGREE) + C(INCOME0) + C(CVD) + CRPRESU0 + BMI0'
lm = smf.ols(formula, data).fit()

In [ ]:
# design matrix...

lm.model.data.orig_exog

In [ ]:
anova = anova_lm(lm)

In [ ]:
anova

In [ ]:
lm.summary()